
# Access dashDB and explore the data with Python

This notebook shows how to access a dashDB data warehouse or DB2 database when using Python. The examples use a dashDB warehouse, but the instructions apply to both dashDB and DB2.

<!--
![dashDB Logo](https://ibm.box.com/shared/static/42yt39czuksqdi278xpy96txtlw3lfmb.png)
![Python Logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM-flattened.png)
-->

## Table of contents

1. [Setup](#Setup) 
1. [Import the *ibmdbpy* Python library](#Import-the-ibmdbpy-Python-library)
1. [Insert the database connection credentials](#Insert-the-database-connection-credentials)
1. [Create the database connection](#Create-the-database-connection)
1. [Use dataframe to read and manipulate tables](#Use-dataframe-to-read-and-manipulate-tables)
1. [Use SQL](#Use-SQL)
1. [Explore the trafficking data](#Explore-the-trafficking-data)
1. [Close the database connection](#Close-the-database-connection)
1. [Summary](#Summary)


## Setup

Before beginning you will need a *dashDB* instance. dashDB is a fully managed cloud data warehouse, purpose-built for analytics. It offers massively parallel processing (MPP) scale and compatibility with a wide range of business intelligence (BI) tools.  

1. __RIGHT-CLICK__ [this link](https://github.com/Davin-IBM/Proof-of-Technology/raw/master/DSX/Lab-1/data/Female-human-trafficking.csv) and select __Save Link As...__ to save the `Female-human-trafficking.csv` file to your desktop.
1. Download the [dashDB setup instructions](https://github.com/Davin-IBM/Proof-of-Technology/raw/master/DSX/Lab-1/dashDB-Setup.pdf).
1. Open up the PDF file that you've just downloaded in the previous step and follow the instructions there to setup dashDB, load the human trafficking data and create a DSX data connection.

<a class="ibm-tooltip" href="https://console.ng.bluemix.net/catalog/services/dashdb" target="_blank" title="" id="ibm-tooltip-0">
<img alt="IBM Bluemix.Get started now" height="193" width="153" src="https://ibm.box.com/shared/static/42yt39czuksqdi278xpy96txtlw3lfmb.png" >
</a>


## Import the *ibmdbpy* Python library

Python support for dashDB and DB2 is provided by the [ibmdbpy Python library](https://pypi.python.org/pypi/ibmdbpy). Connecting to dashDB or DB2 is also enabled by a DB2 driver, libdb2.so.  The ibmdbpy project provides a Python interface for data manipulation and access to in-database algorithms in IBM dashDB and IBM DB2. It accelerates Python analytics by seamlessly pushing operations written in Python into the underlying database for execution, thereby benefitting from in-database performance-enhancing features, such as columnar storage and parallel processing.

### Setup a *JDBC* connection
The JDBC Connection is based on a Java virtual machine. From the ibmdbpy library you can use JDBC to connect to a remote dashDB/DB2 instance. To be able to use JDBC to connect, we need to import the *JayDeBeApi* package and we need to add the dashDB/DB2 data connector jar file to the java class path.

To run a cell, click on the cell and click the run button in the toolbar or press __SHIFT-ENTER__.

Run the following commands to install and load the `JayDeBeApi` package and the `ibmdbpy` library into your notebook:

In [ ]:
#!pip install --trusted-host pypi.python.org JayDeBeApi==0.2.0 --user
#!pip install --trusted-host pypi.python.org --upgrade --user ibmdbpy

In [ ]:
import jaydebeapi
from ibmdbpy import IdaDataBase
from ibmdbpy import IdaDataFrame

In [ ]:
import os
os.environ['CLASSPATH'] = "/usr/local/src/data-connectors-1.4.1/db2jcc4-10.5.0.6.jar"

import jpype
args='-Djava.class.path=%s' % os.environ['CLASSPATH']
jvm = jpype.getDefaultJVMPath()
jpype.startJVM(jvm, args)


## Insert the database connection credentials

Click on the cell below, then on the notebook toolbar, click the box of 1's and 0's, find your database connection and click the __Insert to code__ link under the connection name to have a `credentials_1` dictionary added to the notebook.

__Note__: When this dictionary is inserted, it might have a different name than `credentials_1`.  If so change the dictionary name to `credentials_1` so that the rest of the notebook will work.

Connecting to dashDB requires the following information which are provided by the credentials dictionary inserted:
* Database name 
* Host DNS name or IP address 
* Host port
* Connection protocol
* User ID
* User password

The information `credentials_1` will be used to build a connection string in a subsequent step.

In [ ]:
# @hidden_cell
# The code was removed by DSX for sharing.

## Create the database connection

The following code snippet creates a connection string `connection_string`
and uses the `connection_string` to create a database connection object:


In [ ]:
connection_string = 'jdbc:db2://{hostname}:{port}/{db}:user={username};password={password};'.format(**credentials_1)
idadb = IdaDataBase(dsn=connection_string)

## Use a dataframe to read and manipulate tables

You can now use the connection object `idadb` to query the database:

In [ ]:
df = idadb.show_tables(show_all = True)
df.head(10)

In [ ]:
idadb.exists_table_or_view('GOSALESDW.EMP_EXPENSE_FACT')

Using our previously opened IdaDataBase instance named ‘idadb’, we can open one or several IdaDataFrame objects. They behave like pointers to remote tables.

Let us open the *EMP_EXPENSE_FACT* data set, assuming it is stored in the database under the name ‘GOSALESDW.EMP_EXPENSE_FACT’. The following cell assigns the dataset to a pandas DataFrame.

The [Pandas data analysis library](http://pandas.pydata.org/) provides high-performance, easy-to-use data structures and data analysis tools for the Python programming language. Pandas allows easy processing and manipulation of tabular data, so it is a perfect fit for data extracted from relational databases.


In [ ]:
idadf = IdaDataFrame(idadb, 'GOSALESDW.EMP_EXPENSE_FACT')
idadf.dtypes

You can very easily explore the data in the IdaDataFrame by using built in functions.

Use IdaDataFrame.head to get the first n records of your data set (default 5):

In [ ]:
idadf.head()

Use IdaDataFrame.tail to get the last n records of your data set (default 5):

In [ ]:
idadf.tail(10)

__Note__: Because dashDB operates on a distributed system, the order of rows using IdaDataFrame.head and IdaDataFrame.tail is not guaranteed unless the table is sorted (using an ‘ORDER BY’ clause) or a column is declared as index for the IdaDataFrame (parameter/attribute indexer).

IdaDataFrame also implements most attributes that are available in a pandas DataFrame, here is an example of getting the _shape_ of the data frame (rows, columns):

In [ ]:
idadf.shape

In [ ]:
idadf.columns

Several standard statistics functions from the pandas interface are also available for IdaDataFrame.  You can use the __TAB__ key after the `.` to see the methods and attributes available in an object in a pulldown menu. 

For example, let us calculate various summary statistics for the data set by using the `describe` method for the IdaDataFrame instance:

In [ ]:
idadf.describe()

__Note__: It is possible to subset the rows of an IdaDataFrame by accessing the IdaDataFrame with a slice object. You can also use the IdaDataFrame.loc attribute, which contains an ibmdbpy.Loc object. However, the row selection might be inaccurate if the current IdaDataFrame is not sorted or does not contain an indexer. This is due to the fact that dashDB stores the data across several nodes if available. Moreover, because dashDB is a column oriented database, row numbers are undefined:

In [ ]:
idadf_new = idadf[0:9] # Select the first 10 rows
idadf_new.head(10)

## Use SQL
We can also use SQL to get the top 10.

In [ ]:
top_10_df = idadb.ida_query('SELECT * FROM GOSALESDW.EMP_EXPENSE_FACT LIMIT 10')
top_10_df

Find the top 10 employees (by id) by the number of transactions in the `GOSALESDW.EMP_EXPENSE_FACT` table.

In [ ]:
idadb.ida_query('SELECT EMPLOYEE_KEY, COUNT(*) AS COUNT FROM GOSALESDW.EMP_EXPENSE_FACT GROUP BY EMPLOYEE_KEY ORDER BY COUNT DESC LIMIT 10')

## Explore the trafficking data
__Time to apply what you've learned!__

Complete the tasks that are commented in the code blocks.  If you get stuck, check out the hints.  If you're still stuck, read the solution.  If you have any questions, feel free to ask.

In [ ]:
# Create a variable called `trafficking_table` and set it to the name of the trafficking table in your dashDB instance, for example:
# trafficking_table = 'DASH1234.FEMALE_TRAFFICKING'

trafficking_table = '{}.FEMALE_TRAFFICKING'.format(credentials_1['username']).upper()

idadb.exists_table_or_view(trafficking_table)

# The output of this block should be:
# True

<div class="panel-group" id="accordion-10">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-10" href="#collapse1-10">Hint 1</a></h4>
    </div>
    <div id="collapse1-10" class="panel-collapse collapse">
      <div class="panel-body">
      
The table schema and name can be found [here](#Use-a-dataframe-to-read-and-manipulate-tables).
      
      </div>
    </div>
  </div>
</div>

In [ ]:
# Create an IdaDataFrame called trafficking_df with the trafficking data and determine the shape of the table using the shape atribute of the trafficking_df instance

trafficking_df = IdaDataFrame(idadb, trafficking_table)

print(trafficking_df.shape)
trafficking_df.dtypes

<div class="panel-group" id="accordion-11">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-11" href="#collapse1-11">Hint 1</a></h4>
    </div>
    <div id="collapse1-11" class="panel-collapse collapse">
      <div class="panel-body">
      
It's similar to the code:
<pre>
idadf = IdaDataFrame(idadb, 'GOSALESDW.EMP_EXPENSE_FACT')
</pre>
      
      </div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-11" href="#collapse2-11">Solution</a></h4>
    </div>
    <div id="collapse2-11" class="panel-collapse collapse">
      <div class="panel-body">
<pre>
trafficking_df = IdaDataFrame(idadb, trafficking_table)
</pre>
      </div>
    </div>
  </div>
</div>


In [ ]:
# Show the top 5 records from the table using a method of the IdaDataFrame or alternatively via a SQL query using the ida_query method of the idadb instance

trafficking_df.head()

<div class="panel-group" id="accordion-12">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-12" href="#collapse1-12">Hint 1</a></h4>
    </div>
    <div id="collapse1-12" class="panel-collapse collapse">
      <div class="panel-body">
      
Heads or tails?
      
      </div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-12" href="#collapse2-12">Solution 1</a></h4>
    </div>
    <div id="collapse2-12" class="panel-collapse collapse">
      <div class="panel-body">
      Heads!
<pre>
trafficking_df.head()
</pre> 
      </div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-12" href="#collapse3-12">Solution 2</a></h4>
    </div>
    <div id="collapse3-12" class="panel-collapse collapse">
      <div class="panel-body">
      SQL can work as well!  __Note__: The `+` operator used here is a way to concatenate strings in Python.
<pre>
idadb.ida_query('SELECT * FROM ' + trafficking_table + ' LIMIT 10')
</pre>
      </div>
    </div>
  </div>
</div>


In [ ]:
# Query the trafficking table so that we get the vetting_level and the number of times that vetting_level appears in the trafficking table, ordered by the vetting_level

idadb.ida_query('SELECT VETTING_LEVEL, COUNT(*) AS COUNT FROM ' + trafficking_table + ' GROUP BY VETTING_LEVEL ORDER BY VETTING_LEVEL')


<div class="panel-group" id="accordion-13">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-13" href="#collapse1-13">Hint 1</a></h4>
    </div>
    <div id="collapse1-13" class="panel-collapse collapse">
      <div class="panel-body">
      
It is similar to:

<pre>
idadb.ida_query('SELECT EMPLOYEE_KEY, COUNT(*) AS COUNT FROM GOSALESDW.EMP_EXPENSE_FACT GROUP BY EMPLOYEE_KEY ORDER BY COUNT DESC LIMIT 10')
</pre>
      
      </div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-13" href="#collapse2-13">Hint 2</a></h4>
    </div>
    <div id="collapse2-13" class="panel-collapse collapse">
      <div class="panel-body">
      
The SQL query is:

<pre>
'SELECT VETTING_LEVEL, COUNT(*) AS COUNT FROM ' + trafficking_table + ' GROUP BY VETTING_LEVEL ORDER BY VETTING_LEVEL'
</pre>
      
      </div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title"><a data-toggle="collapse" data-parent="#accordion-13" href="#collapse3-13">Solution</a></h4>
    </div>
    <div id="collapse3-13" class="panel-collapse collapse">
      <div class="panel-body">
      
<pre>
idadb.ida_query('SELECT VETTING_LEVEL, COUNT(*) AS COUNT FROM ' + trafficking_table + ' GROUP BY VETTING_LEVEL ORDER BY VETTING_LEVEL')
</pre>

      </div>
    </div>
  </div>
</div>


## Close the database connection

To ensure expected behaviors, IdaDataBase instances need to be closed.

__Note__: Closing the *IdaDataBase* is equivalent to closing the connection: once the connection is closed, it is no longer possible to use the *IdaDataBase* instance and any *IdaDataFrame* instances that were opened on this connection.

In [ ]:
idadb.close()

## Summary

This notebook demonstrated how to establish a JDBC connection to a dashDB (or DB2) database from Python, work with data frames and explore the data using SQL and data frame object attributes and methods.

## Take it farther
### Free courses on <a href="https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu/" rel="noopener noreferrer" target="_blank">Big Data University</a>: <a href="https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu" rel="noopener noreferrer" target="_blank"><img src = "https://ibm.box.com/shared/static/xomeu7dacwufkoawbg3owc8wzuezltn6.png" width=600px> </a>